In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import json
from tqdm import tqdm
import pandas as pd

import os
import sys
sys.path.append('..')
from utils import nethook
from utils import model_utils

In [3]:
MODEL_NAME = "gpt2-medium"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
mt = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=False)

model = mt.model
tokenizer = mt.tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [4]:
prompt = [
    "This was an awesome movie",
    "This was a bad movie",
    "This was not a good movie",
    "That movie was garbage",
    "Iphone 7 is not a good phone",
    "Google new line of pixels are great"
]

prompt = ["<REVIEW>: " + p + " <SENTIMENT>" for p in prompt]

txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    # debug=True,
    get_answer_tokens=True,

    # track_interesting_words = [
    #     [" positive", " negative"],
    #     [" positive", " negative"]
    # ]
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<REVIEW>: This was an awesome movie <SENTIMENT> <REVIEW>: I love this movie <REVIEW>: I love this movie <REVIEW>: I love this movie <
p(answer):  p(' <'[1279])=0.0664, p('
'[198])=0.0569, p(' :'[1058])=0.05, p(' I'[314])=0.0458, p(' It'[632])=0.0208

<REVIEW>: This was a bad movie <SENTIMENT>
<REVIEW>: This was a bad movie <SENTIMENT> <REVIEW>: I don't know what to say <REVIEW>: I'm not sure what to say <REVIEW
p(answer):  p(' <'[1279])=0.0586, p(' :'[1058])=0.0528, p('
'[198])=0.0445, p(' I'[314])=0.0341, p(' ('[357])=0.0278

<REVIEW>: This was not a good movie <SENTIMENT>
<REVIEW>: This was not a good movie <SENTIMENT> <REVIEW>: I don't think it was a good movie <REVIEW>: I think it was a good movie
p(answer):  p(' <'[1279])=0.0508, p(' :'[1058])=0.0455, p(' I'[314])=0.0397, p('
'[198])=0.0332, p(' ('[357])=0.0229

<REVIEW>: That movie was garbage <SENTIMENT>
<REVIEW>: That movie was garbage <SENTIMENT> <REVIEW>: I'm not going to watch 

## Prompt Tuning

In [6]:
soft_prompts = torch.load("Saved_weights/Promt-Tuned_CLF__IMDB_50K/gpt2-medium/promptuned__epoch_10.pth")

In [9]:
txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    get_answer_tokens=True,
    light_weight_tuning= soft_prompts, algo = "prompt"
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><REVIEW>: This was an awesome movie <SENTIMENT> positive positive positive positive positive
p(answer):  p(' positive'[3967])=0.995, p(' negative'[4633])=0.0049, p(' positively'[19888])=0.0, p(' positives'[38548])=0.0, p(' good'[922])=0.0

<REVIEW>: This was a bad movie <SENTIMENT>
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><REVIEW>: This was a bad movie <SENTIMENT> negative, negative, negative
p(answer):  p(' negative'[4633])=0.9986, p(' positive'[3967])=0.0014, p(' der

## Prefix-tuning

In [17]:
prefix_embeddings = torch.load('Saved_weights/Prefix-Tuned_CLF__IMDB_50K/gpt2-medium/prefixtuned__epoch_1.pth')

In [11]:
txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    get_answer_tokens=True,
    light_weight_tuning= prefix_embeddings, algo = "prefix"
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><REVIEW>: This was an awesome movie <SENTIMENT> positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive
p(answer):  p(' positive'[3967])=0.9853, p(' negative'[4633])=0.0146, p('positive'[24561])=0.0, p(' Positive'[33733])=0.0, p(' favorable'[17070])=0.0

<REVIEW>: This was a bad movie <SENTIMENT>
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><REVIEW>: This was a bad movie <SENTIMENT> negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative
p(answer):  p(' negative'[4633])=0.9859, p(' positive'[3967])=0.014, p('negative'[31591])=0.0, p(' Negative'[36183])=0.0, p(' negatively'

## Adapter Tuning

In [19]:
from torch.nn import Linear, ReLU, Sequential, Tanh

def init_weights(m, lo = -0.0001, hi = 0.0001):
    if isinstance(m, Linear):
        torch.nn.init.uniform_(m.weight, a = lo, b = hi)
        torch.nn.init.uniform_(m.bias, a = lo, b = hi)


class Adapter(torch.nn.Module):
    def __init__(self, inp_out_dim, adapter_dim, hidden_conf = []):
        super().__init__()
        self.inp_out_dim = inp_out_dim
        self.conf = [inp_out_dim] + hidden_conf + [adapter_dim] + hidden_conf[::-1] + [inp_out_dim]
        # print(self.conf)
        self.adapter_dim = adapter_dim
        self.layers = []

        i = 1
        while i < len(self.conf):
            inp = self.conf[i-1]
            out = self.conf[i]
            layer_name = f'layer{i}'
            setattr(self, layer_name, Sequential(Linear(inp, out), ReLU()))
            self.layers.append(layer_name)
            i += 1
    
    def forward(self, x):
        x_init = x.clone()
        for module in self.named_children():
            layer_name = module[0]
            layer = getattr(self, layer_name)
            x = layer(x)
        return x + x_init

In [20]:
adapter_blocks = torch.load("Saved_weights/Adapter-Tuned_CLF__IMDB_50K/gpt2-medium/adapter_tuned__epoch_3.pth")

In [23]:
txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    get_answer_tokens=True,
    light_weight_tuning= adapter_blocks, algo = "adapter"
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<REVIEW>: This was an awesome movie <SENTIMENT> positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive
p(answer):  p(' positive'[3967])=0.5254, p(' awesome'[7427])=0.046, p(' of'[286])=0.0219, p(' great'[1049])=0.0136, p(' negative'[4633])=0.0099

<REVIEW>: This was a bad movie <SENTIMENT>
<REVIEW>: This was a bad movie <SENTIMENT> negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative
p(answer):  p(' negative'[4633])=0.6349, p(' positive'[3967])=0.0435, p(' bad'[2089])=0.0374, p(' of'[286])=0.0193, p(' not'[407])=0.0059

<REVIEW>: This was not a good movie <SENTIMENT>
<REVIEW>: This was not a good movie 

## Fine-tuning

In [ ]:
checkpoint = torch.load('Saved_weights/Fine-Tuned_CLF__IMDB_50K/gpt2-medium/finetuned__epoch_5.pth')
print("loaded checkpoint")
model.load_state_dict(checkpoint)
print("loaded model weights")

In [ ]:
prompt = [
    "This was an awesome movie",
    "This was a bad movie",
    "This was not a good movie",
    "That movie was garbage",
    "Iphone 7 is not a good phone",
    "Google new line of pixels are great"
]

prompt = ["<REVIEW>: " + p + " <SENTIMENT>" for p in prompt]

txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    # debug=True,
    get_answer_tokens=True,

    # track_interesting_words = [
    #     [" positive", " negative"],
    #     [" positive", " negative"]
    # ]
)

model_utils.print_formatted_results(prompt, txt, ret_dict)